## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [2]:
import numpy as np
from datasets import load_dataset, get_dataset_split_names
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import torch
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoFeatureExtractor
from transformers import DistilBertModel, DistilBertTokenizer, EarlyStoppingCallback
from datasets import load_from_disk

#import evaluate
#import wandb

In [9]:
from datasets import load_dataset, get_dataset_split_names

# data loading
dataset_name= "cornell-movie-review-data/rotten_tomatoes"
ds_train = load_dataset(dataset_name, split="train")
ds_test = load_dataset(dataset_name, split="test")
ds_validation = load_dataset(dataset_name, split="validation")

print(ds_train)

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})


In [8]:
import random
idx = random.randint(0, ds_train.shape[0])
print(f"Text example: {ds_train['text'][idx]}")
print(f"Label example: {ds_train['label'][idx]}")

Text example: a graceless , witless attempt at mating some like it hot with the wwii espionage thriller .
Label example: 0


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [10]:
from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("distilbert/distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [11]:
print('######################## TOKENIZER ########################')
print(tokenizer)
print('\n######################## MODEL ########################')
print(model)

######################## TOKENIZER ########################
DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

######################## MODEL ###########

In [ ]:
idx = 5
input = tokenizer(ds_train[idx]['text'], padding=True, return_tensors="pt")
output = model(**input)

print("Samlpe phrase:", ds_train['text'][idx])
print("Token IDs:", input["input_ids"], "\n")
print(output)

Samlpe phrase: the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .
Token IDs: tensor([[  101,  1996,  2143,  3640,  2070,  2307, 12369,  2046,  1996, 11265,
         10976,  4588,  9273,  3388,  1997,  2035,  5888,  1011,  1011,  2130,
          2216,  2040,  2031,  2584,  1996,  7619,  2327,  1997,  1996,  2208,
          1012,   102]]) 

BaseModelOutput(last_hidden_state=tensor([[[-0.1294, -0.1650, -0.0116,  ...,  0.0188,  0.5275,  0.3162],
         [-0.1010, -0.4746,  0.0064,  ...,  0.2497,  1.0458, -0.1916],
         [ 0.2940, -0.0451, -0.2083,  ...,  0.0923,  0.4090,  0.0165],
         ...,
         [ 0.2519, -0.5503,  0.3750,  ...,  0.0188,  0.3540,  0.4099],
         [ 0.8860,  0.0582, -0.1454,  ...,  0.0152, -0.4178, -0.2869],
         [ 0.7707,  0.2344,  0.0241,  ...,  0.0782, -0.2970, -0.0993]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [24]:
idx = 870
input = tokenizer(ds_train[idx]['text'], padding=True, return_tensors="pt")
output = model(**input)

print("Samlpe phrase:", ds_train['text'][idx])
print("Token IDs:", input["input_ids"], "\n")
print(output)

Samlpe phrase: the story , like life , refuses to be simple , and the result is a compelling slice of awkward emotions .
Token IDs: tensor([[  101,  1996,  2466,  1010,  2066,  2166,  1010, 10220,  2000,  2022,
          3722,  1010,  1998,  1996,  2765,  2003,  1037, 17075, 14704,  1997,
          9596,  6699,  1012,   102]]) 

BaseModelOutput(last_hidden_state=tensor([[[-0.1782, -0.1238, -0.0865,  ..., -0.0867,  0.4100,  0.3362],
         [-0.2704, -0.0790, -0.4201,  ...,  0.1000,  0.8274, -0.1355],
         [ 0.0728, -0.2583, -0.1176,  ..., -0.1747,  0.1437, -0.2213],
         ...,
         [-0.4658,  0.1920,  0.0862,  ..., -0.2875,  0.1009, -0.1235],
         [ 0.7563,  0.0812, -0.2714,  ...,  0.3315, -0.5503, -0.4448],
         [-0.5857, -0.1106,  0.3320,  ...,  0.0919,  0.3582, -0.0365]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [26]:
def get_cls_embeddings(texts, tokenizer, model, batch_size=16, device="cpu"):
    """
    Converte una lista di testi in embeddings CLS usando DistilBERT.
    """
    all_embeddings = []
    model.to(device)
    model.eval()

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
        encoded = {k: v.to(device) for k, v in encoded.items()}

        with torch.no_grad():
            outputs = model(**encoded)
            cls_embeddings = outputs.last_hidden_state[:,0,:]  # [CLS]
        
        all_embeddings.append(cls_embeddings.cpu().numpy())

    return np.vstack(all_embeddings)

In [27]:
if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.backends.mps.is_available():
    DEVICE = "mps"
else:
    DEVICE = "cpu"

train_texts, train_labels = ds_train["text"], ds_train["label"]
val_texts, val_labels = ds_validation["text"], ds_validation["label"]
test_texts, test_labels = ds_test["text"], ds_test["label"]

# Estrazione embeddings CLS
X_train = get_cls_embeddings(train_texts, tokenizer, model, device=DEVICE)
X_val = get_cls_embeddings(val_texts, tokenizer, model, device=DEVICE)
X_test = get_cls_embeddings(test_texts, tokenizer, model, device=DEVICE)

In [28]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

clf = LinearSVC()
clf.fit(X_train, train_labels)

val_preds = clf.predict(X_val)
test_preds = clf.predict(X_test)

print("Validation accuracy:", accuracy_score(val_labels, val_preds))
print("Test accuracy:", accuracy_score(test_labels, test_preds))
print("\nClassification report (test):\n", classification_report(test_labels, test_preds))


Validation accuracy: 0.8217636022514071
Test accuracy: 0.798311444652908

Classification report (test):
               precision    recall  f1-score   support

           0       0.79      0.81      0.80       533
           1       0.81      0.78      0.80       533

    accuracy                           0.80      1066
   macro avg       0.80      0.80      0.80      1066
weighted avg       0.80      0.80      0.80      1066



-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [ ]:
# Your code here.

#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [1]:
# Your code here.

#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [2]:
# Your code here.

-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [3]:
# Your code here.

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [4]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [5]:
# Your code here.